In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import cross_val_score as cv
np.random.seed(1)

In [2]:
from sklearn.ensemble import RandomForestRegressor

In [31]:
df_raw = pd.read_clipboard()

In [49]:
(df_raw.notnull().sum()/len(df_raw)).sort_values()

F                       0.859961
name                    1.000000
Wireless Charging       1.000000
Wattage                 1.000000
Ram                     1.000000
Internal                1.000000
Screen to body ratio    1.000000
Size                    1.000000
Single-MP               1.000000
Weight                  1.000000
nanometers              1.000000
#cameras                1.000000
Total_bands             1.000000
CPU_score               1.000000
#2g_bands               1.000000
#4g_bands               1.000000
#5g_bands               1.000000
Announced               1.000000
Price                   1.000000
PPI                     1.000000
Bluetooth               1.000000
VERSION                 1.000000
OS                      1.000000
Battery                 1.000000
Brand                   1.000000
NFC                     1.000000
Thickness               1.000000
#3g_bands               1.000000
GPU_score               1.000000
dtype: float64

In [50]:
df_raw = df_raw[['name', 'Thickness', 'NFC', 'Brand', 'Battery', 'OS', 'VERSION',
       'Bluetooth', 'PPI', 'Price', 'Announced', '#5g_bands', '#4g_bands',
       '#3g_bands', '#2g_bands', 'Total_bands', '#cameras', 'nanometers',
       'Weight', 'Single-MP', 'F', 'Size', 'Screen to body ratio', 'Internal',
       'Ram', 'Wattage', 'Wireless Charging', 'CPU_score', 'GPU_score']]

In [51]:
df_raw = df_raw.fillna(df_raw.median())

In [52]:
(df_raw.notnull().sum()/507).sort_values()

F                       0.859961
name                    1.000000
Wireless Charging       1.000000
Wattage                 1.000000
Ram                     1.000000
Internal                1.000000
Screen to body ratio    1.000000
Size                    1.000000
Single-MP               1.000000
Weight                  1.000000
nanometers              1.000000
#cameras                1.000000
Total_bands             1.000000
CPU_score               1.000000
#2g_bands               1.000000
#4g_bands               1.000000
#5g_bands               1.000000
Announced               1.000000
Price                   1.000000
PPI                     1.000000
Bluetooth               1.000000
VERSION                 1.000000
OS                      1.000000
Battery                 1.000000
Brand                   1.000000
NFC                     1.000000
Thickness               1.000000
#3g_bands               1.000000
GPU_score               1.000000
dtype: float64

In [53]:
x = df_raw[['Thickness', 'NFC', 'Brand', 'Battery','VERSION',
           'Bluetooth', 'PPI', 'Announced',
           '#5g_bands', '#4g_bands', '#3g_bands', '#2g_bands', 'Total_bands',
           '#cameras', 'nanometers','Weight', 'Single-MP',
           'Size', 'Screen to body ratio', 'Internal','Ram',
           'Wattage', 'Wireless Charging', 'CPU_score', 'GPU_score']]

In [54]:
x.dtypes

Thickness               float64
NFC                       int64
Brand                    object
Battery                   int64
VERSION                   int64
Bluetooth               float64
PPI                     float64
Announced               float64
#5g_bands                 int64
#4g_bands                 int64
#3g_bands                 int64
#2g_bands                 int64
Total_bands               int64
#cameras                  int64
nanometers              float64
Weight                  float64
Single-MP               float64
Size                    float64
Screen to body ratio    float64
Internal                  int64
Ram                     float64
Wattage                 float64
Wireless Charging         int64
CPU_score               float64
GPU_score               float64
dtype: object

In [55]:
x = pd.get_dummies(x)

In [62]:
test_idx = pd.Series(range(len(x))).sample(int(len(x)/4), random_state=42).tolist()
train_idx = [i for i in range(len(x)) if i not in test_idx]

In [64]:
X_test = x.iloc[test_idx]
X_train = x.iloc[train_idx]

scaler = StandardScaler()
X_scaled_tr = scaler.fit_transform(X_train)
X_scaled = StandardScaler().fit_transform(x)
X_scaled_ts = scaler.transform(X_test)

C:\Users\Synergy\AppData\Local\conda\conda\envs\threeseven\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Synergy\AppData\Local\conda\conda\envs\threeseven\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Synergy\AppData\Local\conda\conda\envs\threeseven\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Synergy\AppData\Local\conda\conda\envs\threeseven\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  re

In [65]:
y_test = df_raw.Price.iloc[test_idx].values
y_train = df_raw.Price.iloc[train_idx].values

In [66]:
rf = RandomForestRegressor()
rf.fit(X = X_scaled_tr, y = y_train)
preds_tr = rf.predict(X_scaled_tr)
preds_ts = rf.predict(X_scaled_ts)
print((abs(preds_tr - y_train)/y_train).mean(), (abs(preds_ts - y_test)/y_test).mean())
print(np.median(abs(preds_tr - y_train)/y_train), np.median(abs(preds_ts - y_test)/y_test))
print(r2_score(preds_tr,y_train), r2_score(preds_ts,y_test))

0.12904304652811785 0.3681521592763541
0.0764444444444444 0.2113312074829933
0.8349427505064883 0.23600410461489274


C:\Users\Synergy\AppData\Local\conda\conda\envs\threeseven\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
